In [ ]:
import os
from neo4j import GraphDatabase

URI = "bolt://localhost:7687"  # Replace with your Neo4j URI
AUTH = ("neo4j", "password")   # Replace with your Neo4j username and password

driver = GraphDatabase.driver(URI, auth=AUTH)

In [ ]:
def import_json_to_neo4j(tx, json_file_path):
    query = """
    CALL apoc.load.json($file_path)
    YIELD value AS data
    UNWIND data.businesses AS business_json // Adjust based on your JSON structure
    MERGE (business:Business {id: business_json.business_id})
    SET business += business_json

    UNWIND data.users AS user_json // Adjust based on your JSON structure
    MERGE (user:User {id: user_json.user_id})
    SET user += user_json

    // Example: Creating relationships from reviews (adjust based on your JSON)
    UNWIND data.reviews AS review_json
    MERGE (review:Review {id: review_json.review_id})
    SET review += review_json

    MERGE (user_node:User {id: review_json.user_id})
    MERGE (business_node:Business {id: review_json.business_id})
    MERGE (user_node)-[:WROTE_REVIEW]->(review)-[:REVIEWS]->(business_node)
    """
    tx.run(query, file_path=json_file_path)


with driver.session() as session:
    json_file_path = "file:///path/to/your/data.json" # Replace with the actual path to your JSON file (within Neo4j import directory or accessible via URL)
    session.execute_write(import_json_to_neo4j, json_file_path=json_file_path)

driver.close()